<a href="https://colab.research.google.com/github/SDY-Github/Audio_Spectrogram/blob/main/audio_spectrogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
#https://github.com/sikora507/elgen/blob/master/src/audio%20analysis.ipynb

import tensorflow as tf
import os
import numpy as np
cwd = os.getcwd()

In [44]:
cd /content/drive/MyDrive/audio_spectrogram/audio

/content/drive/MyDrive/audio_spectrogram/audio


In [45]:
# numpy 상위 라이브러리 scipy

import scipy.io.wavfile

In [46]:
"""
scipy.io.wavfile.read
: Return the sample rate (in samples/sec) and data from an LPCM WAV file.
-> rate : 가져온 파일의 sample/sec audData = data
"""

path = "./"
file_list = os.listdir(path)
wav_list = [file for file in file_list if file.endswith(".wav")]

print ("wav_list: {}".format(wav_list))

rate_list = [0 for i in range(len(wav_list))] # 각 wav 파일의 rate 저장할 리스트 -> rate : int
audData_list = [0 for i in range(len(wav_list))] # 각 wav 파일의 Data 저장할 리스트 audData : (XXX,XXX : 2) 배열 (1열 left, 2열 right)

for i in range(len(wav_list)) :
  #print(wav)
  rate_list[i],audData_list[i]=scipy.io.wavfile.read(wav_list[i])

print(rate_list) # rate는 wav 파일마다 다름
print()
print(audData_list[0].shape[0])

wav_list: ['SnowVillage_p308.wav', 'AbovetheTreetops_p002.wav', 'Nautilus_p238.wav', 'DragonRider_p096.wav', 'QueensGarden_p265.wav', 'Nautilus238.wav', 'DragonRider096.wav', 'AbovetheTreetops002.wav', 'SnowVillage308.wav', 'QueensGarden265.wav']
[48000, 48000, 48000, 48000, 48000, 44100, 44100, 22050, 22050, 22050]

9232380


In [47]:
# 첫번째 저장된 wav 파일 정보 확인
print(audData_list[0].shape) # [audData 개수(길이) : left&right] 구조
print(audData_list[0][0]) # 첫번째 audData 값 [left right]
print(audData_list[0]) # 전체 auData 값
print(audData_list[0].dtype) # datatype
print(rate_list[0]) # wav파일의 rate (sample/sec)

(9232380, 2)
[0 1]
[[ 0  1]
 [ 0 -2]
 [ 0  3]
 ...
 [ 2  2]
 [-1 -3]
 [-1  3]]
int16
48000


In [48]:
# 가져온 audData의 1열 : 왼쪽 소리 / 2열 : 오른쪽 소리
channel1_list = [0 for i in range(len(wav_list))]
channel2_list = [0 for i in range(len(wav_list))]

for i in range (len(wav_list)) :
  channel1_list[i]=audData_list[i][:,0] #left
  channel2_list[i]=audData_list[i][:,1] #right

print(channel1_list)
print(channel1_list[0].shape[0])

[array([ 0,  0,  0, ...,  2, -1, -1], dtype=int16), array([ 1, -3,  3, ...,  0, -1,  1], dtype=int16), array([ 0,  0,  0, ...,  0,  1, -1], dtype=int16), array([ 0,  1, -2, ...,  1, -2,  1], dtype=int16), array([0, 0, 0, ..., 1, 0, 0], dtype=int16), array([0, 0, 0, ..., 0, 0, 0], dtype=int16), array([-1, -1,  0, ...,  0,  0,  0], dtype=int16), array([0, 0, 0, ..., 0, 0, 0], dtype=int16), array([0, 0, 0, ..., 0, 0, 0], dtype=int16), array([0, 0, 0, ..., 0, 0, 0], dtype=int16)]
9232380


In [51]:
def list_chunk(ls, S) : # ls 리스트를 S 단위로 자름
  return [ls[i:i+S] for i in range (0, len(ls), S)]

# 노래의 일부 자르기!!! 왼쪽 채널만 사용
signal_fragment_list = [0 for i in range(len(wav_list))]
N = 10 # 노래 나눌 길이 (sec)

#for i in range (len(wav_list)) :
#  signal_fragment_list[i] = channel1_list[i][30*rate_list[i]:35*rate_list[i]]

for i in range (len(wav_list)) :
  sec = channel1_list[i].shape[0] / rate_list[i]
  print(sec)
  signal_fragment_list[i] = list_chunk(channel1_list[i], N*rate_list[i])

"""
print(signal_fragment_list)
print(len(signal_fragment_list))
print(len(signal_fragment_list[0][19]))
"""
print(len(signal_fragment_list[2]))


192.34125
268.42466666666667
130.85589583333334
140.16091666666668
130.51354166666667
122.25412698412698
132.85984126984127
271.3360090702948
192.75968253968253
119.53845804988661
14


### Create FFT

In [38]:
import numpy as np
import math
from PIL import Image
import time

In [39]:
FFT_LENGTH = 1024
WINDOW_LENGTH = 512
WINDOW_STEP = int(WINDOW_LENGTH / 2)
magnitudeMin = float("inf")
magnitudeMax = float("-inf")
phaseMin = float("inf")
phaseMax = float("-inf")

In [40]:
def amplifyMagnitudeByLog(d):
    return 188.301 * math.log10(d + 1)

def weakenAmplifiedMagnitude(d):
    return math.pow(10, d/188.301)-1

def generateLinearScale(magnitudePixels, phasePixels, 
                        magnitudeMin, magnitudeMax, phaseMin, phaseMax):
    height = magnitudePixels.shape[0]
    width = magnitudePixels.shape[1]
    magnitudeRange = magnitudeMax - magnitudeMin
    phaseRange = phaseMax - phaseMin
    rgbArray = np.zeros((height, width, 3), 'uint8')
    
    for w in range(width):
        for h in range(height): # 각 픽셀마다 진폭 , 위상 정보 가짐
            magnitudePixels[h,w] = (magnitudePixels[h,w] - magnitudeMin) / (magnitudeRange) * 255 * 2
            magnitudePixels[h,w] = amplifyMagnitudeByLog(magnitudePixels[h,w])
            phasePixels[h,w] = (phasePixels[h,w] - phaseMin) / (phaseRange) * 255
            # 이미지 rgb에 magnitude 저장
            red = 255 if magnitudePixels[h,w] > 255 else magnitudePixels[h,w]
            green = (magnitudePixels[h,w] - 255) if magnitudePixels[h,w] > 255 else 0
            blue = phasePixels[h,w]
            rgbArray[h,w,0] = int(red)
            rgbArray[h,w,1] = int(green)
            rgbArray[h,w,2] = int(blue)
    return rgbArray

def recoverLinearScale(rgbArray, magnitudeMin, magnitudeMax, 
                       phaseMin, phaseMax):
    width = rgbArray.shape[1]
    height = rgbArray.shape[0]
    magnitudeVals = rgbArray[:,:,0].astype(float) + rgbArray[:,:,1].astype(float)
    phaseVals = rgbArray[:,:,2].astype(float)
    phaseRange = phaseMax - phaseMin
    magnitudeRange = magnitudeMax - magnitudeMin
    for w in range(width):
        for h in range(height):
            phaseVals[h,w] = (phaseVals[h,w] / 255 * phaseRange) + phaseMin
            magnitudeVals[h,w] = weakenAmplifiedMagnitude(magnitudeVals[h,w])
            magnitudeVals[h,w] = (magnitudeVals[h,w] / (255*2) * magnitudeRange) + magnitudeMin
    return magnitudeVals, phaseVals

In [41]:
def generateSpectrogramForWave(signal): # 잘라온거 받음
    start_time = time.time()
    global magnitudeMin
    global magnitudeMax
    global phaseMin
    global phaseMax
    buffer = np.zeros(int(signal.size + WINDOW_STEP - (signal.size % WINDOW_STEP)))
    buffer[0:len(signal)] = signal
    height = int(FFT_LENGTH / 2 + 1)
    width = int(len(buffer) / (WINDOW_STEP) - 1)
    magnitudePixels = np.zeros((height, width))
    phasePixels = np.zeros((height, width))

    for w in range(width): # w : 이미지 가로 픽셀
        buff = np.zeros(FFT_LENGTH)
        stepBuff = buffer[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH]
        # apply hanning window
        stepBuff = stepBuff * np.hanning(WINDOW_LENGTH)
        buff[0:len(stepBuff)] = stepBuff
        #buff now contains windowed signal with step length and padded with zeroes to the end
        fft = np.fft.rfft(buff)
        for h in range(len(fft)):
            #print(buff.shape)
            #return
            magnitude = math.sqrt(fft[h].real**2 + fft[h].imag**2) # fft로 magnitude 계산
            if magnitude > magnitudeMax:
                magnitudeMax = magnitude 
            if magnitude < magnitudeMin:
                magnitudeMin = magnitude 

            phase = math.atan2(fft[h].imag, fft[h].real) # phase = frequency?
            if phase > phaseMax:
                phaseMax = phase
            if phase < phaseMin:
                phaseMin = phase
            magnitudePixels[height-h-1,w] = magnitude # 진폭 정보
            phasePixels[height-h-1,w] = phase # 위상 정보
    rgbArray = generateLinearScale(magnitudePixels, phasePixels,
                                  magnitudeMin, magnitudeMax, phaseMin, phaseMax) # rgb형태로 생성
    elapsed_time = time.time() - start_time
    print('%.2f' % elapsed_time, 's', sep='')
    img = Image.fromarray(rgbArray, 'RGB')
    return img


In [54]:
for i in range (len(wav_list)) : # 모든 노래
  for j in range (len(signal_fragment_list[i])) : # 각 노래 자른 개수
    img = generateSpectrogramForWave(signal_fragment_list[i][j]) # signal_fragment : 일정부분 잘라온거
    #scipy.io.wavfile.write("/content/drive/MyDrive/audio_analysis/audio/" + , rate, signal_fragment) # (filname, rate, data) wav 파일 새로 씀 (굳이 할필요 X)
    img.save("/content/drive/MyDrive/audio_spectrogram/spectrogram/" + wav_list[i][:-4] + "_" + str(j+1) + ".png","PNG") # *.wav 이름으로 저장된 wav_list에서 .png로 이름 변경하여 저장

#generateSpectrogramForWave(signal_fragment_list)

9.45s
9.54s
9.26s
9.17s
9.29s
9.55s
9.23s
9.17s
9.21s
9.37s
9.23s
9.26s
9.29s
9.37s
9.23s
9.32s
9.34s
9.22s
9.28s
2.17s
9.42s
9.36s
9.30s
9.37s
9.28s
9.26s
9.22s
9.32s
9.45s
9.43s
9.12s
9.39s
9.39s
9.88s
9.58s
10.13s
9.75s
9.60s
9.27s
9.60s
9.72s
9.72s
9.58s
9.70s
9.50s
9.53s
7.88s
9.56s
9.63s
9.36s
9.24s
9.37s
9.40s
9.65s
9.22s
9.26s
9.31s
9.36s
9.27s
9.27s
0.80s
9.33s
9.17s
9.28s
9.37s
9.32s
9.20s
9.20s
9.21s
9.22s
9.24s
9.23s
9.21s
9.28s
9.15s
0.14s
9.32s
9.29s
9.07s
9.35s
9.30s
9.45s
9.17s
9.33s
9.34s
9.38s
9.26s
9.13s
9.32s
0.48s
8.44s
8.48s
8.63s
8.70s
8.40s
8.44s
8.49s
8.48s
8.49s
8.45s
8.54s
8.59s
1.93s
8.56s
8.59s
8.55s
8.44s
8.66s
8.64s
8.71s
8.53s
8.64s
8.54s
8.67s
8.39s
8.56s
2.44s
4.30s
4.23s
4.23s
4.23s
4.29s
4.33s
4.32s
4.28s
4.31s
4.25s
4.27s
4.47s
4.34s
4.20s
4.27s
4.28s
4.39s
4.28s
4.22s
4.25s
4.32s
4.30s
4.24s
4.34s
4.26s
4.22s
4.23s
0.57s
4.32s
4.19s
4.22s
4.26s
4.31s
4.23s
4.17s
4.21s
4.16s
4.19s
4.24s
4.30s
4.34s
4.24s
4.22s
4.25s
4.28s
4.31s
4.21s
1.17s
4.32s
4.2

### Recover from png

In [58]:
cd /content/drive/MyDrive/audio_spectrogram/spectrogram/

/content/drive/MyDrive/audio_spectrogram/spectrogram


In [59]:
# spectrogram 디렉토리에 있는 png 파일 읽어옴

path = "./"
file_list = os.listdir(path)
png_list = [file for file in file_list if file.endswith(".png")]

print ("png_list: {}".format(png_list))

png_list: ['SnowVillage_p308.png', 'AbovetheTreetops_p002.png', 'Nautilus_p238.png', 'DragonRider_p096.png', 'QueensGarden_p265.png', 'Nautilus238.png', 'DragonRider096.png', 'AbovetheTreetops002.png', 'SnowVillage308.png', 'QueensGarden265.png', 'SnowVillage_p308_1.png', 'SnowVillage_p308_2.png', 'SnowVillage_p308_3.png', 'SnowVillage_p308_4.png', 'SnowVillage_p308_5.png', 'SnowVillage_p308_6.png', 'SnowVillage_p308_7.png', 'SnowVillage_p308_8.png', 'SnowVillage_p308_9.png', 'SnowVillage_p308_10.png', 'SnowVillage_p308_11.png', 'SnowVillage_p308_12.png', 'SnowVillage_p308_13.png', 'SnowVillage_p308_14.png', 'SnowVillage_p308_15.png', 'SnowVillage_p308_16.png', 'SnowVillage_p308_17.png', 'SnowVillage_p308_18.png', 'SnowVillage_p308_19.png', 'SnowVillage_p308_20.png', 'AbovetheTreetops_p002_1.png', 'AbovetheTreetops_p002_2.png', 'AbovetheTreetops_p002_3.png', 'AbovetheTreetops_p002_4.png', 'AbovetheTreetops_p002_5.png', 'AbovetheTreetops_p002_6.png', 'AbovetheTreetops_p002_7.png', 'Abov

In [60]:
def recoverSignalFromSpectrogram(filePath,rate):
    img = Image.open(filePath)
    data = np.array( img, dtype='uint8' )
    width = data.shape[1]
    height = data.shape[0]

    magnitudeVals, phaseVals \
    = recoverLinearScale(data, magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
    recovered = np.zeros(WINDOW_LENGTH * width // 2 + WINDOW_STEP, dtype=np.int16)
    for w in range(width):
        toInverse = np.zeros(height, dtype=np.complex_)
        for h in range(height):
            magnitude = magnitudeVals[height-h-1,w]
            phase = phaseVals[height-h-1,w]
            toInverse[h] = magnitude * math.cos(phase) + (1j * magnitude * math.sin(phase))
        signal = np.fft.irfft(toInverse)
        recovered[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH] += signal[:WINDOW_LENGTH].astype(np.int16)
    scipy.io.wavfile.write(filePath[:-4] + "_recovered.wav", rate, recovered) # spectrogram 읽은 디렉토리에 복원된 음원 저장

In [ ]:
# 읽어온 png 파일에 대해 음원으로 복원

for i in range (len(wav_list)) :
  recoverSignalFromSpectrogram("/content/drive/MyDrive/audio_spectrogram/spectrogram/" + png_list[i], rate_list[i])